In [2]:
import pandas as pd
import argparse
import time
import csv
import pickle
from itertools import zip_longest
import operator
import datetime
import os
from collections import Counter
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from io import StringIO
import matplotlib.pyplot as plt

In [2]:
purpose = "train_only" #"train_only", "train_test", "test_only"

In [3]:
user1 = 'A' # train
user2 = 'B' # test
user1_name = user1
user2_name = user2
user1 = pd.read_pickle(user1+'.gzip')
user2 = pd.read_pickle(user2+'.gzip')

In [4]:
user1

,ecount,SID,SCount,CID,time,CMD,whole_cmd
0,1,2016-02-23_17:07:00_saguaro2_pts-0,1,1,2016-02-23 17:07:04,ll,ll
1,1,2016-02-23_17:07:00_saguaro2_pts-0,1,2,2016-02-23 17:07:13,top,top
2,1,2016-02-23_17:07:00_saguaro2_pts-0,1,3,2016-02-23 17:07:30,ll,ll
3,1,2016-02-23_17:07:00_saguaro2_pts-0,1,4,2016-02-23 17:07:38,mkdir,mkdir D3cavity
4,1,2016-02-23_17:07:00_saguaro2_pts-0,1,5,2016-02-23 17:07:39,ll,ll
...,...,...,...,...,...,...,...
639670,1,2022-01-13_18:35:56_agave2_pts-39,4829,187989,2022-01-17 18:47:24,ls,ls
639671,1,2022-01-13_18:35:56_agave2_pts-39,4829,187990,2022-01-17 18:47:29,ll,ll
639672,1,2022-01-13_18:35:56_agave2_pts-39,4829,187991,2022-01-17 18:47:55,cd,cd ..
639673,1,2022-01-13_18:35:56_agave2_pts-39,4829,187992,2022-01-17 18:58:13,cd,cd freq0.010_freq0.299/


In [5]:
#Converting the timestamp to UNIX time
import datetime
import time

def f(x):
    date, tm = str(x).split(" ")
    year, month, date = date.split("-")
    hr, mn, sec = tm.split(":")
    #regular python datetime
    date_time = datetime.datetime(int(year), int(month), int(date), int(hr), int(mn), int(sec))
    #Returning unix timestamp after conversion
    return int(time.mktime(date_time.timetuple()))

user1['time'] = user1['time'].apply(f)
user2['time'] = user2['time'].apply(f)

In [6]:
#Taking the date from SID column
def f(x):
    return x[:10]

user1['SID'] = user1['SID'].apply(f)
user2['SID'] = user2['SID'].apply(f)

In [7]:
#Renumbering of the sessions
if list(user1['SCount'])[-1] == list(set(list(user1['SCount'])))[-1]:
    print("Go on with renumbering sessions from 1!")
    print("If your SCount column starts from some number other than 1!")
    
user1['SCount'] = user1['SCount'] - (list(user1['SCount'])[0]-1)
user2['SCount'] = user2['SCount'] - (list(user2['SCount'])[0]-1)

Go on with renumbering sessions from 1!
If your SCount column starts from some number other than 1!


In [8]:
user2

,ecount,SID,SCount,CID,time,CMD,whole_cmd
0,2,2018-02-20,1,1,1519173190,ll,ll
2,1,2018-02-20,2,3,1519173329,ll,ll
5,2,2018-02-20,3,5,1519181212,ll,ll
6,1,2018-02-20,3,6,1519181220,mkdir,mkdir document
7,1,2018-02-20,3,7,1519181221,ll,ll
...,...,...,...,...,...,...,...
198854,1,2020-05-11,1018,196827,1589230086,cd,cd sigma0.010
198855,1,2020-05-11,1018,196828,1589230087,ls,ls
198856,1,2020-05-11,1018,196829,1589230094,cat,cat job1.sh
198857,1,2020-05-11,1018,196830,1589230375,ll,ll


In [9]:
#Label encode the commands
le = LabelEncoder()
le.fit(user1['CMD'])
user1['CMD'] = le.transform(user1['CMD'])
le_length = user1['CMD'].nunique()

In [10]:
user2_cmd = []
for cmd in tqdm(list(user2['CMD'])):
    try:
        user2_cmd += [le.transform([cmd])[0]]
    except:
        user2_cmd += [le_length+1]
user2['CMD'] = user2_cmd

100%|██████████| 197842/197842 [01:37<00:00, 2019.95it/s]


In [11]:
print("Unique commands lenth for user1: %d, user2: %d"%(user1['CMD'].nunique(), user2['CMD'].nunique()))

Unique commands lenth for user1: 1290, user2: 193


In [12]:
#Let's merge the sessions
user2['SCount'] = user2['SCount'] + list(user1['SCount'])[-1]

In [13]:
#Let's merge two dataframes
merged_users = pd.concat([user1, user2])

In [14]:
merged_users

,ecount,SID,SCount,CID,time,CMD,whole_cmd
0,1,2016-02-23,1,1,1456272424,796,ll
1,1,2016-02-23,1,2,1456272433,1176,top
2,1,2016-02-23,1,3,1456272450,796,ll
3,1,2016-02-23,1,4,1456272458,885,mkdir D3cavity
4,1,2016-02-23,1,5,1456272459,796,ll
...,...,...,...,...,...,...,...
198854,1,2020-05-11,5847,196827,1589230086,540,cd sigma0.010
198855,1,2020-05-11,5847,196828,1589230087,828,ls
198856,1,2020-05-11,5847,196829,1589230094,528,cat job1.sh
198857,1,2020-05-11,5847,196830,1589230375,796,ll


In [15]:
len(set(list(merged_users['CMD'])))

1291

In [16]:
len(set(list(merged_users['SCount'])))

5335

In [17]:
# merged_users.to_csv('merged_users.csv')

# Data Preprocessing

In [18]:
session_split = int(list(user1['SCount'])[-1])
dataset='merged_users.csv'

In [29]:
reader = StringIO()
merged_users.to_csv(reader)
reader.seek(0)

# reader = csv.DictReader(f, delimiter=',')
sess_commands = {}
sess_date = {}
ctr = 0
curid = -1
curdate = None
columns = {'':0, 'ecount':1, 'SID':2, 'SCount':3, 'CID':4, 'time':5, 'CMD':6, 'whole_cmd':7}
for data in list(csv.reader(reader))[1:]:
    sessid = data[columns['SCount']] #Session number
    if curdate and not curid == sessid:

        date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
        sess_date[curid] = date
    curid = sessid #Current session id 

    command = data[columns['CMD']], int(data[columns['time']]) #The command id and timeframe of that command
    curdate = data[columns['SID']] #Date of the session

    if sessid in sess_commands:
        sess_commands[sessid] += [command]
    else:
        sess_commands[sessid] = [command]
    ctr += 1

date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
sess_date[curid] = date

for i in list(sess_commands):
    sorted_clicks = sorted(sess_commands[i], key=operator.itemgetter(1)) #Sorting the commands based on timeframe
    sess_commands[i] = [c[0] for c in sorted_clicks]

print(len(list(sess_commands)))


def lensOfSessions():
    return [len(sess_commands[s]) for s in list(sess_commands)]

print(len(sess_commands))
# Removing the same commands in sequence of 2 length
for s in list(sess_commands):
    sess_commands[s] = [i for i, j in zip_longest(sess_commands[s], sess_commands[s][1:]) if i != j]

# Filter out length 1 sessions
for s in list(sess_commands):
    if len(sess_commands[s]) == 1 or len(sess_commands[s])>=300:
        del sess_commands[s]
        del sess_date[s]

print("Number of sessions after filtering 1 length sessions:", len(sess_commands))

print("Avg Length of sessions: ", np.mean(lensOfSessions()))
print(Counter(lensOfSessions()))

# Count number of times each command appears
commandid_counts = {}
for s in sess_commands:
    if int(s) <= session_split:
        seq = sess_commands[s]
        for commandid in seq:
            if commandid in commandid_counts:
                commandid_counts[commandid] += 1
            else:
                commandid_counts[commandid] = 1

sorted_counts = sorted(commandid_counts.items(), key=operator.itemgetter(1))
# print(sorted_counts)
# Filtering commands appearing less than 5 times
# Try this when you only consider the first word in one command
length = len(sess_commands)
print("\nLength of sess_commands before filtering less than 5 command counts:",len(sess_commands))
dates = list(sess_date.items())
tra_sess = filter(lambda x: int(x[0]) <= int(session_split), dates)
tes_sess = filter(lambda x: int(x[0]) > int(session_split), dates)
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))
print("\n1Length of training sessions sorted:", len(tra_sess))
print("1Length of testing sessions sorted:", len(tes_sess))
for s in list(sess_commands):
    if int(s)<=session_split:
        curseq = sess_commands[s]
        filseq = list(filter(lambda i: commandid_counts[i] <= 5000 and commandid_counts[i] >= 2, curseq))
        if len(filseq) < 2 and int(s)<=session_split:
    #         print(curseq, filseq)
            del sess_commands[s]
            del sess_date[s]
        else:
            sess_commands[s] = filseq
print("Length of sess_commands after filtering less than 5 command counts:",len(sess_commands))
dates = list(sess_date.items())
tra_sess = filter(lambda x: int(x[0]) <= int(session_split), dates)
tes_sess = filter(lambda x: int(x[0]) > int(session_split), dates)
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))
print("\n2Length of training sessions sorted:", len(tra_sess))
print("2Length of testing sessions sorted:", len(tes_sess))

# Split out test set based on dates
dates = list(sess_date.items())

print('Splitting sessions:', session_split)
tra_sess = filter(lambda x: int(x[0]) <= int(session_split), dates)
tes_sess = filter(lambda x: int(x[0]) > int(session_split), dates)

# Sort sessions by date
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))

print(len(tes_sess))

print("\nLength of training sessions sorted:", len(tra_sess))
print("Length of testing sessions sorted:", len(tes_sess))
print("\nFirst three training sessions:", tra_sess[:3])
print("First three test sessions:", tes_sess[:3])

print("-- Splitting done: train set and test set @ %ss" % datetime.datetime.now())

command_dict = {}
command_ctr = 1
# Convert training sessions to sequences and renumber commands to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    global command_ctr
    for s, date in tra_sess:
        seq = sess_commands[s] #The sequence of commands in this session
        outseq = []
        for i in seq:
            if i in command_dict:
                outseq += [command_dict[i]]
            else:
                outseq += [command_ctr]
                command_dict[i] = command_ctr
                command_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print("\nNumber of unique commands (after all the filtering):", command_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs

# Convert test sessions to sequences, ignoring commands that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_commands[s]
        outseq = []
        for i in seq:
            if i in command_dict:
                outseq += [command_dict[i]]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs

tra_ids, tra_dates, tra_seqs = obtian_tra() #Train session id/numbers, Train session dates, Train session sequence
tes_ids, tes_dates, tes_seqs = obtian_tes() #Test session id/numbers, Test session dates, Test session sequence

def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]
            ids += [id]
    return out_seqs, out_dates, labs, ids

print("\nLength of training sessions:", len(tra_seqs))
print("Length of testing sessions:", len(tes_seqs))
tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)

print(np.mean(np.array([len(i) for i in tr_seqs])))
# print(Counter([len(i) for i in tr_seqs]))

# tot_seqs = tr_seqs + te_seqs
# tot_labs = tr_labs + te_labs
# tot_seqs = np.asarray(tot_seqs, dtype=object)
# tot_labs = np.asarray(tot_labs, dtype=object)
# if True:
#     shuffled_arg = np.arange(len(tot_seqs))
#     np.random.shuffle(shuffled_arg)
#     tot_seqs = tot_seqs[shuffled_arg]
#     tot_labs = tot_labs[shuffled_arg]

# split = int(len(tot_labs)*0.9)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)

print("\nLength of new training sequences derived from all the sessions:", len(tra[0]))
print("Length of new testing sequences derived from all the sessions:", len(tes[0]))
# print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
# print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('Average length of sequence in the dataset training and testing combined: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

print('Done.')


5335
5335
Number of sessions after filtering 1 length sessions: 4446
Avg Length of sessions:  57.52069275753486
Counter({2: 172, 3: 161, 6: 131, 5: 115, 7: 114, 4: 112, 8: 103, 10: 98, 15: 97, 11: 88, 9: 87, 12: 81, 14: 73, 13: 73, 17: 71, 19: 63, 20: 60, 16: 57, 18: 56, 23: 53, 21: 53, 27: 52, 28: 51, 22: 48, 26: 45, 30: 44, 29: 42, 25: 41, 24: 40, 32: 38, 34: 37, 36: 37, 33: 36, 35: 36, 40: 32, 44: 30, 45: 29, 47: 28, 38: 28, 43: 28, 57: 27, 42: 27, 31: 26, 50: 25, 56: 25, 49: 24, 46: 24, 62: 21, 53: 21, 65: 21, 41: 21, 37: 21, 76: 20, 51: 20, 77: 20, 54: 20, 73: 20, 48: 19, 39: 19, 55: 18, 64: 18, 52: 17, 140: 17, 130: 17, 66: 16, 59: 16, 78: 16, 81: 16, 70: 16, 74: 16, 79: 16, 67: 15, 60: 15, 71: 15, 83: 14, 68: 14, 75: 14, 108: 14, 84: 13, 110: 13, 102: 12, 63: 12, 128: 12, 124: 11, 93: 11, 80: 11, 69: 11, 72: 11, 120: 11, 111: 11, 61: 11, 119: 11, 89: 11, 122: 11, 95: 11, 101: 11, 105: 10, 82: 10, 125: 10, 99: 10, 103: 10, 96: 10, 116: 10, 86: 10, 148: 10, 90: 10, 104: 10, 145: 1

In [20]:
command_ctr

435

# Training

In [21]:
import os
os.chdir('../pytorch_code/')

In [22]:
import argparse
import pickle
import time
from utils import build_graph, Data, split_validation
from model import *

import json
import torch
import multiprocessing as mp
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F
torch.cuda.is_available()

/data/sse/.local/opt/conda/envs/shellprint/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [27]:
best_result = [0,0,0]
best_epoch = [0,0,0]
bad_counter = 0

def train(epoch, model, train_data):
    #print('start training: ', datetime.datetime.now())
    model.train()
    total_loss = [0.0]
    slices = train_data.generate_batch(model.batch_size)
    pbar = tqdm(zip(slices, np.arange(len(slices))))
    train_hit, train_mrr, train_acc = [], [], []
    for i, j in pbar:
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
#         print(targets.shape, scores.shape)
        
        train_sub_scores = scores.topk(5)[1]
        train_sub_scores = trans_to_cpu(train_sub_scores).detach().numpy()
#         print(np.array(train_sub_scores).shape)
        
        for score, target, mask in zip(train_sub_scores, targets, train_data.mask):
            train_hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                train_mrr.append(0)
            else:
                train_mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
#                 print(score, target)
#                 print((np.where(score == target - 1)[0][0] + 1))
            
            if score[0]==target-1: train_acc += [1]
            else: train_acc+=[0]
        
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += [loss.item()]
        #f j % int(len(slices) / 5 + 1) == 0:
        pbar.set_description('Epoch %d:[%d/%d] Train Loss:%.3f Train Accuracy:%.3f Train Precision@5:%.3f Train MRR@5:%.3f' % (epoch, j, len(slices), np.mean(total_loss), np.mean(train_acc), np.mean(train_hit), np.mean(train_mrr)))
#     model.scheduler.step()
#         print(np.array(train_hit).shape)
    
    train_hit = np.mean(train_hit) * 100
    train_mrr = np.mean(train_mrr) * 100
    train_acc = np.mean(train_acc) * 100
    train_loss = np.mean(total_loss)
    
    return train_hit, train_mrr, train_acc, train_loss

def test(model, test_data):
    
    total_loss = [0.0]
    slices = test_data.generate_batch2(model.batch_size, 400)
    print(np.array(slices).shape)
    pbar = tqdm(zip(slices, np.arange(len(slices))))
    results = []
    
    for i, j in pbar:
        hit, mrr, acc = [], [], []
        targets, scores = forward(model, i, test_data)
#         print(np.array(targets).shape, np.array(scores).shape)
        sub_scores = scores.topk(5)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
#         print(np.array(sub_scores).shape)
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
#                 print((np.where(score == target - 1)[0][0] + 1))

            if score[0]==target-1: acc += [1]
            else: acc+=[0]
        
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        total_loss += [loss.item()]
        
        results.append([loss.item(), np.mean(acc), np.mean(hit), np.mean(mrr)])
        
        pbar.set_description('[%d/%d] Test Loss:%.3f Test Accuracy:%.3f Test Precision@5:%.3f Test MRR@5:%.3f' % (j, len(slices), np.mean(total_loss), np.mean(acc), np.mean(hit), np.mean(mrr)))
#         print(np.array(hit).shape)
    res = pd.DataFrame(results, columns=['test_loss', 'test_acc', 'test_hit', 'test_mrr'])
    res.to_csv("Test.csv")
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    acc = np.mean(acc) * 100
    loss = np.mean(total_loss)
    
    return hit, mrr, acc, loss

In [24]:
purpose = 'test_only'

In [25]:
train_data = Data(tra, shuffle=True)
test_data = Data(tes, shuffle=True)

class opt:
        def __init__(self):
            self.batchSize=200
            self.dataset='sample'
            self.epoch=100
            self.hiddenSize=100
            self.l2=1e-05
            self.lr=0.001
            self.lr_dc=0.1
            self.lr_dc_step=3
            self.n_node=232
            self.nonhybrid=False
            self.patience=10
            self.step=1
            self.valid_portion=0.1
            self.validation=False
opt=opt()

17402it [00:00, 125114.68it/s]
100%|██████████| 17402/17402 [00:00<00:00, 195586.80it/s]
3961it [00:00, 875738.66it/s]
100%|██████████| 3961/3961 [00:00<00:00, 496492.68it/s]


In [28]:
if purpose=='train_only':
    
    model = SessionGraph(opt, command_ctr)
    model = model.cuda()
    
    for epoch in range(opt.epoch):

        train_hit, train_mrr, train_acc, train_loss = train(epoch, model, train_data)
        print("Train Loss:%.4f Train Accuracy:%.4f Train Precision@5:%.4f Train MRR@5:%.4f\n"% (train_loss, train_acc, train_hit, train_mrr))
        flag = 0
        if train_hit >= best_result[0]:
            best_result[0] = train_hit
            best_epoch[0] = epoch
            flag = 1
        if train_mrr >= best_result[1]:
            best_result[1] = train_mrr
            best_epoch[1] = epoch
            flag = 1
        if train_acc >= best_result[2]:
            best_result[2] = train_acc
            best_epoch[2] = epoch
            flag = 1

    print('Best Result:')
    print('\tRecall@20:\t%.4f\tMRR@5:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))

    model_name = user1_name+'_model.pth'
    print('Saving final model states to {}...'.format('model/'+model_name))
    torch.save(model.state_dict(), '../model/'+model_name)

elif purpose=='train_test':
    results = []
    model = SessionGraph(opt, command_ctr)
    model = model.cuda()
    
    for epoch in range(opt.epoch):

        train_hit, train_mrr, train_acc, train_loss = train(epoch, model, train_data)
        print("Train Loss:%.4f Train Accuracy:%.4f Train Precision@5:%.4f Train MRR@5:%.4f\n"% (train_loss, train_acc, train_hit, train_mrr))
        model.eval()
        test_hit, test_mrr, test_acc, test_loss = test(model, test_data)
        print("Test Loss:%.4f Test Accuracy:%.4f Test Precision@5:%.4f Test MRR@5:%.4f\n"% (test_loss, test_acc, test_hit, test_mrr))
        results.append([train_loss, train_acc, train_hit, train_mrr, test_loss, test_acc, test_hit, test_mrr])
        flag = 0
        if train_hit >= best_result[0]:
            best_result[0] = train_hit
            best_epoch[0] = epoch
            flag = 1
        if train_mrr >= best_result[1]:
            best_result[1] = train_mrr
            best_epoch[1] = epoch
            flag = 1
        if train_acc >= best_result[2]:
            best_result[2] = train_acc
            best_epoch[2] = epoch
            flag = 1

    res = pd.DataFrame(results, columns=['train_loss', 'train_acc', 'train_hit', 'train_mrr', 'test_loss', 'test_acc', 'test_hit', 'test_mrr'])
    
    print('Best Result:')
    print('\tRecall@20:\t%.4f\tMRR@5:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))

    model_name = user1_name+'_model.pth'
    print('Saving final model states to {}...'.format('model/'+model_name))
    torch.save(model.state_dict(), '../model/'+model_name)
    res.to_csv('../model/'+user1_name+'_'+user2_name+'_model.csv')
    
elif purpose=='test_only':
    test_model = SessionGraph(opt, command_ctr)
    test_model = test_model.cuda()
    
    test_model_name = user1_name+'_model.pth'
    test_model.load_state_dict(torch.load('../model/'+test_model_name))
    test_model.eval()

    test_hit, test_mrr, test_acc, test_loss = test(test_model, test_data)

    print("Test Loss:%.4f Test Accuracy:%.4f Test Precision@5:%.4f Test MRR@5:%.4f\n"% (test_loss, test_acc, test_hit, test_mrr))
#     except:
#         print("Can't find the model!")

else:
    print("Choose the right option.")

(400, 200)


[399/400] Test Loss:2.503 Test Accuracy:0.465 Test Precision@5:0.775 Test MRR@5:0.583: : 400it [00:37, 10.73it/s]

Test Loss:2.5031 Test Accuracy:46.5000 Test Precision@5:77.5000 Test MRR@5:58.2833



In [60]:
import numpy as np
import random

a = [1,2,3,4,5,6,7,8,9,10,11,34]
random.choices(a, k=3)

[6, 9, 34]

In [56]:
np.split(np.arange(10), 2)

[array([0, 1, 2, 3, 4]), array([5, 6, 7, 8, 9])]

In [ ]:
%matplotlib inline

filename = '../model/'+user1_name+'_'+user2_name+'_model.csv'

history = pd.read_csv(filename, sep=',')
'train_loss', 'train_acc', 'train_hit', 'train_mrr', 'test_loss', 'test_acc', 'test_hit', 'test_mrr'
train_acc = history['train_acc']
train_hit = history['train_hit']
train_loss = history['train_loss']
train_mrr = history['train_mrr']

test_acc = history['test_acc']
test_hit = history['test_hit']
test_loss = history['test_loss']
test_mrr = history['test_mrr']

epochs = range(opt.epoch)

plt.figure(figsize=(16,16))
plt.subplot(2,2,1)
plt.plot(epochs, train_acc, 'b', label='Train Accuracy')
plt.plot(epochs, test_acc, 'r', label='Test Accuracy')
plt.title('Accuracy')
plt.legend(loc=0)

plt.subplot(2,2,2)
plt.plot(epochs, train_hit, 'b', label='Train Precision@5')
plt.plot(epochs, test_hit, 'r', label='Test Precision@5')
plt.title('Precision@5')
plt.legend(loc=0)

plt.subplot(2,2,3)
plt.plot(epochs, train_mrr, 'b', label='Train MRR@5')
plt.plot(epochs, test_mrr, 'r', label='Test MRR@5')
plt.title('MRR@5')
plt.legend(loc=0)

plt.subplot(2,2,4)
plt.plot(epochs, train_loss, 'b', label='Train Loss')
plt.plot(epochs, test_loss, 'r', label='Test Loss')
plt.title('Loss')
plt.legend(loc=0)


plt.savefig(filename[:-10]+'.png')
plt.show()